# Compliance zone for two MBS panels
**Author**: Dr Vitas Anderson (*Two Fields Consulting*)

**Last updated**: 28/10/2023

+ The aim of this notebook is to test an algorithm for creating a point spatial representation of a spatially averaged compliance zone
+ FEKO data provided by Danie Ludick
+ Uses **v11** RFcalcUC module
+ This notebook uses dataframes in HDF5 files that were generated from the `make_HDF5_MBS_panel_trial2.ipynb` notebook

## Load necessary python modules

In [1]:
# import mayavi mlab
from traits.etsconfig.api import ETSConfig
ETSConfig.toolkit = 'qt4'  # the 'qt4' option actually denotes qt4 and qt5
from mayavi import mlab

# import other modules
from importlib import reload
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
import pandas as pd
import RFcalcUC_v11 as RFcalcUC
from matplotlib.ticker import StrMethodFormatter
pd.set_option('display.max_rows', 200)
from icecream import ic

# Set plotting environment
sn.set()

In [2]:
# Create function to show exclsuion zone and PSSACZ as 3D scatter plot points
def ExclusionZone(self, data, power, bg='lightgrey', fg='black',
                  color=['green','blue','orange','magenta'],
                  alpha=[0.5]*8, setting=['public']*8, standard=['RPS-S1']*8, 
                  title='', axv=[True,False,False], ycut=None, zcut=None,
                  hman=None, xyzman=[-1.5,0,0], 
                  antenna=RFcalcUC.panelAntenna, antdz=None, 
                  gridpoints=False, gridpoint_filter=None, gridpoint_size=0.02, gridpoint_opacity=0.3,
                  figsize=(1200,900), pssacz=False):
    '''
        Draw Mayavi figures of exclusion zones for datasets in S
                S = dataframe for x,y,z
             data = list of S data sets, e.g.['Smax','SE','SH','SARwb']
            power = list of scaled power levels for data, e.g. [200,200,100]
               bg = background color, selected from COLORS dictionary, e.g. 'white'
               fg = foreground color, selected from COLORS dictionary, e.g. 'black'
            color = list of colours for exclusion zones for data, e.g. ['red','blue','crimson']
            alpha = opacity of the exclusion zone [0 to 1]
          setting = list of settings for data, e.g. ['pub','occ']
         standard = EME exposure standard for limit value ['RPS S-1 WB','FCC']
            title = displayed title for the plot
              axv = list or tuple indication visibility of x,y,z axes, e.g. (False,False,True) for just z axis visible
             ycut = y value to set cutplane where all points have y > ycut [ystart to yend]
             zcut = z value to set cutplane where all points have z > zcut [zstart to zend]
             hman = height of man figure
           xyzman = [x,y,z] coords of centre of man
       gridpoints = toggle to display gridpoint. Defau;ti is False to NOT display gridpoints
 gridpoint_filter = Mask for gridpoints. Default is valid points for 1.6m spatial averaging
   gridpoint_size = size of gridpoints. Default is 0.02
gridpoint_opacity = opacity of the gripoints [0 to 1]
          antenna = function for display of antenna
            antdz = displacement of second antenna in vertical direction. Enter None to not display
       gridpoints = toggle for displaying gridpoints
          figsize = tuple of width and height f figure in pixels, e.g. (1200,900)
           pssacz = boolean flag to display the point spatial compliance zone
        '''

    # make sure that data, power, color, setting, standard are lists or tuples
    # and have at least as many elements as in data
    def makeIterable(arg, argname):
        if isinstance(arg, (str,float,int)):  # make sure that single string inputs are contained in a list
            arg = [arg]
        if argname != 'data':
            errmsg = f'{argname} must have at least as many elements as in data {data}'
            assert len(arg) >= len(data), errmsg 
        return arg

    data = makeIterable(data,'data')
    power = makeIterable(power,'power')
    color = makeIterable(color,'color')
    alpha = makeIterable(alpha,'alpha')
    setting = makeIterable(setting,'setting')
    standard = makeIterable(standard,'standard')
    COLORS = RFcalcUC.COLORS
    Slimit = RFcalcUC.Slimit

    # assertion tests
    Scols = [col for col in self.S.columns if col not in ('x','y','z','r','phi')]  # i.e. all columns except the x y z r phi coordinates
    for d in data:
        assert d in Scols, f"data ({d}) must be one of {Scols}"    
    for c in color:
        assert c in COLORS.keys(), f"color ({c}) must be in {list(COLORS.keys())}"
    for a in alpha:
        assert (a >= 0) and (a <= 1), f"alpha ({a}) must be in [0,1] range."
    for p in power:
        assert p >= 0, f"power ({p}) must be >= 0"
    if hman != None:
        assert 0.5 <= hman <= 3, f"hman ({hman}) must be between 0.5 and 3"
    assert bg in COLORS.keys(), f"bg {bg} must be one of {COLORS.keys()}"
    assert fg in COLORS.keys(), f"fg {fg} must be one of {COLORS.keys()}"
    
        
    # Background and foreground colors
    bgc = COLORS[bg]
    fgc = COLORS[fg]

    # Calculate the S and SAR limits
    limits, limtexts = [], []
    for dat, s, std in zip(data, setting, standard):
        if 'SAR' in dat:
            limit = SARlimit(s)
            limtext = f'{limit} W/kg'
            limits.append(limit)
            limtexts.append(limtext)
        else:
            limit = Slimit(self.freq, s, std)
            limtext = f'{limit:0.1f} W/m²'
            limits.append(limit)
            limtexts.append(limtext)

    # Calculate the contour level of each exclusion zone
    contour = [self.power / p * lim for p, lim in zip(power, limits)]
    
    # Calculate X, Y, Z mgrid
    X = self.xm
    Y = self.ym
    Z = self.zm
    
    if ycut is not None:
        ystart, yend, dy = self.grid['y']
        assert ystart < ycut < yend, f"ycut ({ycut} must lie within the grid's y value range [{ystart}, {yend}])"
        nc = int((ycut - ystart) / dy)
        X = X[:,nc:,:]
        Y = Y[:,nc:,:]
        Z = Z[:,nc:,:]
        
    if zcut is not None:
        zstart, zend, dz = self.grid['z']
        assert zstart < zcut < zend, f"zcut ({zcut} must lie within the grid's z value range [{zstart}, {zend}])"
        nc = int((zcut - zstart) / dz)
        X = X[:,:,nc:]
        Y = Y[:,:,nc:]
        Z = Z[:,:,nc:]

    # Calculate the x,y,z extents of the plot for all exclusion zones
    ScbAll = pd.DataFrame(columns=['x','y','z']).astype(float)
    for dat, con in zip(data,contour):
        print(f'{dat=}, {con=}')
        mask = (self.S[dat] >= con) & (self.S[dat] < 1.1*con)
        Scb = self.S.loc[mask,['x','y','z']]
        # ScbAll = ScbAll.append(Scb)
        ScbAll = pd.concat([ScbAll, Scb])
        
    extent = ScbAll.apply([min,max]).T.values.flatten().round(1).tolist()
    print(extent)

    # create the Mayavi figure
    try:
        fig = mlab.figure(1, size=figsize, bgcolor=bgc, fgcolor=fgc)
        mlab.clf()
    except:
        raise Exception("Could not create mlab figure")

    # draw the iso-surfaces
    titles = [] if title == '' else [title + '\n']
    for dat,limtext,p,col,a,s,std,con in zip(data,limtexts,power,color,alpha,setting,standard,contour):
        print(f'power={self.power}, plotpower={p}, setting={s}, limit={limtext}, contour level={con:0.3f}')
        t = f'{col.upper()}: {std} {s} exclusion zone ({limtext}) for {p} W {self.datatitles[dat]}'
        titles.append(t)
        S = self.make_mgrid(dat)
        if ycut is not None:
            S = S[:,nc:,:]
        if zcut is not None:
            S = S[:,:,nc:]
        S = np.nan_to_num(S, nan=0.0)  # replace nans in S with zeros
        if S.min() < con < S.max():    # check that con lies within range of values in S field
            try:
                src = mlab.pipeline.scalar_field(X, Y, Z, S, name=dat)
                mlab.pipeline.iso_surface(src, contours=[con, ], opacity=a,
                                          color=COLORS[col])
            except:
                raise Exception(f"Could not draw iso-surface for {dat}")

    # Draw points inside the PSSACZ
    if pssacz:
        Ss = self.S[self.S.inside_pssacz]
        mlab.points3d(Ss.x.values, Ss.y.values, Ss.z.values, color=COLORS['darkblue'],
                      scale_factor=0.05, scale_mode='none')       

    # display gridpoints
    if gridpoints == True:
        pointcolor = COLORS['blue']
        if gridpoint_filter == None:
            # create mask for valid 1.6m spatial averaging points
            gridpoint_filter = self.sf('spatavg', offset=0.001, spatavgL=1.6)
        Sp = self.S[gridpoint_filter.mask]
        if ycut != None:
            Sp = Sp[Sp.y >= ycut]
        extent = Sp[['x','y','z']].apply([min,max]).T.values.flatten().round(1).tolist()
        mlab.points3d(Sp.x.values,Sp.y.values,Sp.z.values,
                      scale_factor=gridpoint_size,color=pointcolor,
                      opacity=gridpoint_opacity)

    # draw the axes
    ax = mlab.axes(x_axis_visibility=axv[0], y_axis_visibility=axv[1],
              z_axis_visibility=axv[2], line_width=1, extent=extent)            
    ax.label_text_property.color = fgc
    ax.title_text_property.color = fgc
    ax.axes.label_format = '%g'
    ax.axes.font_factor = 1

    # print plot title
    height, size = 0.08, 0.08
    mlab.title('\n'.join(titles), height=height, size=size)

    # draw the antenna
    antenna('blue')
    
    # draw second elevated antenna
    if antdz != None:
        antenna('blue',z0=antdz)
    
    # draw the man figure
    if hman != None:
        xc, yc, zc = xyzman
        mlabman(hman, xc, yc, zc)
        
    # Set up the mayavi view
    mlab.view(90, -90)  # xz plane (azimuth=90°, elevation=90°)
    fig.scene.parallel_projection = True
    mlab.show()
    return

## Create RFc object, R, for single panel antenna

### Import field data for a single panel antenna

In [3]:
# Read in "calc uc trials.xls" spreadsheet
trials = pd.read_excel('../antennas/spat avg trials.xlsx', 'trials', 
                       skiprows=1, index_col=0).fillna('')
# trials.drop("unit", axis=1, inplace=True)

# Set trial number
trial = 2
t = trials[str(trial)]

# Set data and plot folders
datadir = f'../antennas/{t.antenna_folder}/data/'
plotdir = f'../antennas/{t.antenna_folder}/plots/'

# Specify the RFc object parameters
freq = t.fMHz   # 900 MHz
power = t.Prad  # 80 W

# Field point grid
grid = dict(x=[t.xstart, t.xend, t.dx], 
            y=[t.ystart, t.yend, t.dy],
            z=[t.zstart, t.zend, t.dz])

# Antenna box dimensions (xyz extents)
xb = [-0.04, 0]
yb = [-0.15, 0.15]
zb = [-1.125, 1.125]
antennabox = [xb, yb, zb]

# Create the RFc object
reload(RFcalcUC)
R = RFcalcUC.RFc(freq, power, grid, antennabox, spatavgL=1.6)

# Import the S data files for FEKO and IXUS
Sfile = datadir + f'{t.antenna_folder}.hdf5'
R.importS(Sfile)

# Create filters for valid 1.6m and 2m spatial averaging points
fsa16 = R.sf('spatavg', spatavgL=1.6) # valid 1.6m spatial averaging points
foutant = R.sf('outant', offset=0)    # all points outside antenna box

# Display the first few records in the S dataframe
display(R)
display(R.S.head())

Sfile='../antennas/MBSpanel_2/data/MBSpanel_2.hdf5'


Object parameters:
  900 MHz, 200 W
  errtol = 0.15
  offset = 0 m
  default length of spatial averagering window = 1.6 m
  1,298,751 grid points
  nx = 151, ny = 141, nz = 61

,x,y,z,r,phi,SE,SH,Smax,SARwb,SARwbi
0,-1.0,-7.0,-3.0,7.071068,-98.130102,0.007138,0.007219,0.007219,NaN,NaN
1,-1.0,-7.0,-2.9,7.071068,-98.130102,0.007635,0.007725,0.007725,NaN,NaN
2,-1.0,-7.0,-2.8,7.071068,-98.130102,0.007646,0.007729,0.007729,NaN,NaN
3,-1.0,-7.0,-2.7,7.071068,-98.130102,0.007189,0.007244,0.007244,NaN,NaN
4,-1.0,-7.0,-2.6,7.071068,-98.130102,0.006404,0.006415,0.006415,NaN,NaN


In [4]:
# Show xyz grid extents for single panel antenna fields
R.grid

{'x': [-1, 14, 0.1], 'y': [-7, 7, 0.1], 'z': [-3, 3, 0.1]}

In [5]:
# 3D view of public exclusion zone for single 200W panel antenna
R.ExclusionZone(
    data=["Smax"],
    power=[200],
    color=["gold"],
    alpha=[0.5],
    setting=["public"],
    standard=["RPS S-1 WB"],
    title="RPS S-1 public exclusion zone (Smax) for single panel antenna at 200W",
    figsize=(1200,1200),
    axv=[True,False,True]
)

dat='Smax', con=4.5
[-0.1, 12.8, -5.8, 5.8, -1.3, 1.3]
power=200, plotpower=200, setting=public, limit=4.5 W/m², contour level=4.500


## Create R2 for combined fields of two panel antennas
The second additional panel antenna is displaced by dz = 3m from the first and has 60% of its radiated power

In [5]:
# Set vertical distance between antennas
antdz = 3  # distance between antenna centres
p2 = 0.6   # power proportion of p2 wrt p1, i.e. p2/p1 

# Create R1 as deep copy of R
import copy
R1 = copy.deepcopy(R)
dropcols = ['SE','SH','SARwb','phi','r']
R1.S.drop(columns=dropcols,inplace=True)

# Create R2 as deepy copy of R1 and offset z by antdz
R2 = copy.deepcopy(R1)
R2.S['z'] = R2.S.z + antdz

# Ensure compatibility of R2.S.z with R1.S.z for later merge operation
R2.S['z'] = R2.S.z.round(1).astype(float)

# Get the field data columns
datcols = R1.S.loc[:,"Smax":].columns.tolist()

# merge S for R1 and R2 and fill nulls with 0
S2 = pd.merge(R1.S,R2.S,'outer',on=['x','y','z'],suffixes=('_1','_2')).sort_values(['x','y','z'])
S2 = S2.fillna(0)

# Sum field levels of merged datcols in S2
for col in datcols:
    col1 = col + '_1'
    col2 = col + '_2'
    S2[col] = S2[col1] + p2 * S2[col2]

# Sort S2 by x,y,z
S2 = S2.sort_values(['x','y','z'])
# display(S2[S2.Smax.isnull()])
    
# Get rid of  _1 and _2 columns
cols = [col for col in S2.columns if '_' not in col]
S2 = S2[cols]

# Update R2 with S2 and other self variables relevant to the new enlarged S dataframe
R2.S = S2
R2.nz = len(S2.z.unique())
R2.grid = dict(x=[t.xstart, t.xend, t.dx], 
               y=[t.ystart, t.yend, t.dy],
               z=[t.zstart, t.zend+antdz, t.dz])
R2.xm = R2.make_mgrid('x')
R2.ym = R2.make_mgrid('y')
R2.zm = R2.make_mgrid('z')

R2.S.head()

,x,y,z,Smax,SARwbi
0,-1.0,-7.0,-3.0,0.007219,0.0
1,-1.0,-7.0,-2.9,0.007725,0.0
2,-1.0,-7.0,-2.8,0.007729,0.0
3,-1.0,-7.0,-2.7,0.007244,0.0
4,-1.0,-7.0,-2.6,0.006415,0.0


In [7]:
# 3D view of Smax public exclusion zone for two panel antennas
R2.ExclusionZone(
    data=["Smax"],
    power=[200],
    color=["gold"],
    alpha=[0.5],
    setting=["public"],
    standard=["RPS S-1 WB"],
    figsize=(1200,1200),
    title="RPS S-1 public exclusion zone (Smax) for two panel antennas at 200W",
    bg='white',
    axv=[True,True,True],
    antdz=antdz
)

dat='Smax', con=4.5
[-0.1, 12.9, -5.8, 5.8, -1.3, 4.2]
power=200, plotpower=200, setting=public, limit=4.5 W/m², contour level=4.500


## Create spatially averaged fields for R2
Create some new S columns for vertical spatial averaging

**c** is the convolution vector applied to z columns of points for each (x,y) group

In [6]:
def conv_vector(l, dl, m):
    '''Create a convolution vector
        l = length of convolution vector (m)
       dl = increment spacing between points in convolution vector (m)
        m = number of averaging points in convolution vector'''

    n = int(round(l/dl)) + 1  # number of points in convolution vector'''
    c = np.ones(n)            # convolution vector
    ix = np.arange(n)         # index of convolution vector
    dix = (n-1)/(m-1)         # index spacing between convolution averaging points

    assert round(l/dl,0) == round(l/dl,1), f'dl ({dl}) does not divide equally into l ({l})'
    assert type(m) == int, f'm ({m}) must be an integer'
    assert (n-1) % (m-1) == 0, f'm ({m}) does not distribute evenly across number of convolution points ({n})'

    c[(ix % dix) != 0] = 0
    c = c/m

    assert np.isclose(sum(c), 1, rtol=1e-08), f'sum of c ({sum(c)}) does not add up to 1'
    
    return c

# 1.6m vertical average curves
l, dl = 1.6, 0.1
for m in [17]:
    c = conv_vector(l, dl, m)
    print(f'\n{m:2d} points: sum{list(c.round(3))} = {sum(c)}')
    R2.spatavg(f"Smax-{l}m-{m}",  "Smax",  c, title=f"FEKO Smax levels ({m} points averaged over {l}m)")

R2.datatitles


17 points: sum[0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059, 0.059] = 1.0
creating Smax-1.6m-17 spatial average of Smax


{'SE': 'S(E) levels',
 'SH': 'S(H) levels',
 'Smax': 'Smax levels',
 'SARps': 'peak spatial SAR',
 'SARwb': 'WBA SAR',
 'SARwbi': 'WBA SAR (interpolated)',
 'Smax-1.6m-17': 'FEKO Smax levels (17 points averaged over 1.6m)'}

In [7]:
R2.S.head(2)

,x,y,z,Smax,SARwbi,Smax-1.6m-17
0,-1.0,-7.0,-3.0,0.007219,0.0,0.003416
1,-1.0,-7.0,-2.9,0.007725,0.0,0.003911


In [10]:
# 3D view of spatially averaged Smax-1.6m-17 public exclusion zone for two panel antennas
R2.ExclusionZone(
    data=["Smax-1.6m-17"],
    power=[200],
    color=["gold"],
    alpha=[0.5],
    setting=["public"],
    standard=["RPS S-1 WB"],
    figsize=(1200,1200),
    title="RPS S-1 public exclusion zone (Smax-1.6m-17) for two panel antennas at 200W",
    bg='white',
    axv=[True,True,False],
    antdz=antdz,
)

dat='Smax-1.6m-17', con=4.5
[0.0, 11.6, -4.6, 4.6, -1.8, 4.9]
power=200, plotpower=200, setting=public, limit=4.5 W/m², contour level=4.500


## Point spatial representation of spatial avg compliance zone

### Create function to detect if point is inside PSSACZ
+ PSSACZ = point spatial spatial average compliance zone
+ Sbot = S value at the bottom of the spatial averaging window
+ Smid = S value in the middle of the spatial averaging window
+ Stop = S value at the top of the spatial averaging window

In [8]:
# Set parameters
Si = 'Smax-1.6m-17'
l = 1.6      # spatial averaging length
dz = 0.1     # dz increment between grid points
Slim = 4.5   # S limit level

# Calculate the number of grid point gaps in half a spatial averaging length
ngaps = int(round(l/dz)/2)
print(f'{ngaps=}')

# Initialise the global boolean mask for whether the point is inside the PSSACZ 
inside_pssacz = np.empty(0).astype(bool)

# Calculate the inside_pssacz mask for each combination of x & y and add to global mask 
# mask = (R2.S.x==3) & (R2.S.y==0)
for name, g in R2.S.groupby(['x','y']):
    
    # Create array for Smid
    Smid = g[Si].values

    # Create arrays for Sbot & Stop by rollong the Smid array indexes up and down
    Sbot = np.roll(Smid,ngaps)
    Stop = np.roll(Smid,-ngaps)

    # Create masks for Sbot, Smid, Stop > Slim
    mask_bot = Sbot > Slim
    mask_mid = Smid > Slim
    mask_top = Stop > Slim

    # Identify points inside the pssacz by first finding where all of Sbot, Smid and Stop are > Slim (case 8)
    inside_pssacz_xy = mask_bot & mask_mid & mask_top

    # Next, recognise that where Smid < Slim, then all points in the averaging window are outside the pssacz   
    ix_mid = np.where(~mask_mid)[0]   # get array indexes where Smid < Slim
    # print(f'{len(inside_pssacz_xy)}')
    # print(f'{ix_mid=}')
    for ix in ix_mid:
        inside_pssacz_xy[(ix-ngaps):(ix+ngaps)] = False
    # print(f'{inside_pssacz_xy=}')
    
    # Lastly, identify inside pts where (Sbot < Slim) & (Smid > Slim) & (Stop < Slim) & (Sbot ~ Stop) (case 3b)
    mask_3b = np.isclose(Sbot, Stop, rtol=0.1)
    inside_pssacz_xy = inside_pssacz_xy | (~mask_bot & mask_mid & ~mask_top & mask_3b)

    # display(pd.DataFrame({'z':g.z.values,'Sbot':Sbot,'Smid':Smid,'Stop':Stop,'inside_PSCCZ_xy':inside_pssacz_xy}))

    # Concatenate the inside_pssacz_xy mask to the global mask
    inside_pssacz = np.concatenate([inside_pssacz,inside_pssacz_xy], axis=0)

R2.S['inside_pssacz'] = inside_pssacz

ngaps=8


In [11]:
# 3D view of spatially averaged Smax-1.6m-17 public exclusion zone for two panel antennas
ExclusionZone(
    self = R2,
    data=["Smax-1.6m-17"],
    power=[200],
    color=["gold"],
    alpha=[0.5],
    setting=["public"],
    standard=["RPS S-1 WB"],
    figsize=(1200,1200),
    title="RPS S-1 public exclusion zone (Smax-1.6m-17) for two panel antennas at 200W",
    bg='white',
    axv=[True,True,False],
    antdz=antdz,
    pssacz=True
)

dat='Smax-1.6m-17', con=4.5
[0.0, 11.6, -4.6, 4.6, -1.8, 4.9]
power=200, plotpower=200, setting=public, limit=4.5 W/m², contour level=4.500


In [15]:
# 3D view of spatially averaged Smax-1.6m-17 public exclusion zone for two panel antennas
ExclusionZone(
    self = R2,
    data=["Smax"],
    power=[200]*2,
    color=["red","gold"],
    alpha=[0.3,0.3],
    setting=["public"]*2,
    standard=["RPS S-1 WB"]*2,
    figsize=(1200,1200),
    title="RPS S-1 public exclusion zone (Smax-1.6m-17) for two panel antennas at 200W",
    bg='white',
    axv=[True,True,False],
    antdz=antdz,
    pssacz=True
)

dat='Smax', con=4.5
[-0.1, 12.9, -5.8, 5.8, -1.3, 4.2]
power=200, plotpower=200, setting=public, limit=4.5 W/m², contour level=4.500


## Export R2.S

In [14]:
# R2.S.to_csv('PSSACZ data file.csv.zip',index=False)
R2.S.head()

,x,y,z,Smax,SARwbi,Smax-1.6m-17,inside_pssacz
0,-1.0,-7.0,-3.0,0.007219,0.0,0.003416,False
1,-1.0,-7.0,-2.9,0.007725,0.0,0.003911,False
2,-1.0,-7.0,-2.8,0.007729,0.0,0.004615,False
3,-1.0,-7.0,-2.7,0.007244,0.0,0.005593,False
4,-1.0,-7.0,-2.6,0.006415,0.0,0.006896,False


## Scratch

In [80]:
RFcalcUC.COLORS

{'blue': (0.2549019607843137, 0.4117647058823529, 0.8823529411764706),
 'darkblue': (0, 0, 0.55),
 'green': (0, 1, 0),
 'darkgreen': (0, 0.55, 0),
 'red': (1, 0, 0),
 'darkred': (0.55, 0, 0),
 'coral2': (0.9333333333333333, 0.41568627450980394, 0.3137254901960784),
 'yellow': (1, 1, 0),
 'gold': (1, 0.8431372549019608, 0),
 'black': (0, 0, 0),
 'orange': (1, 0.5490196078431373, 0),
 'magenta': (1, 0, 1),
 'white': (1, 1, 1),
 'pink': (1, 0.75, 0.8),
 'brown': (0.35, 0.24, 0.12),
 'olive': (0.29, 0.33, 0.13),
 'lightgrey': (0.8706, 0.8706, 0.8706),
 'lightgrey2': (0.34, 0, 34, 0, 34),
 'darkgrey': (0.1, 0.1, 0.1)}

In [ ]:
Create a dictionary for the [Sbottom, S0, Stop] > Slim boolean against 
whether the point is inside the PSSACZ
inside_pssacz = {
    : False,  # case 1 [False,False,False]
    : False,   # case 2 [False,False,True]
    : 'Case3', # case 3 [False,True,False]
    : False,    # case 4 [False,True,True]
    : False,   # case 5 [True,False,False]
    : False,    # case 6 [True,False,True]
    : False,    # case 7 [True,True,False]
    : True       # case 8
}

In [83]:
aa = np.array([1,2,3,4,5,6])
display(aa)
ix = np.where(aa <= 3)[0]
display(ix)
aa[ix+1] = aa[ix]+10
aa[ix]

array([1, 2, 3, 4, 5, 6])

array([0, 1, 2], dtype=int64)

array([ 1, 11, 12])

In [86]:
aa = np.array([1,2,3,4,5,6])
i = 2
aa[i-1:i+1]

array([2, 3])